<a href="https://colab.research.google.com/github/y2o2u2n/boostcourse-dl-tensorflow/blob/master/06_softmax_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
tf.set_random_seed(777)  # for reproducibility
tfe = tf.contrib.eager


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [4]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

# convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

# num classes
nb_classes = 3

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [5]:
# Weight and bias setting
W = tfe.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tfe.Variable(tf.random_normal([nb_classes]), name='bias')
variables = [W, b]

print(W,b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 0.7706481 ,  0.37335402, -0.05576323],
       [ 0.00358377, -0.5898363 ,  1.5702795 ],
       [ 0.2460895 , -0.09918973,  1.4418385 ],
       [ 0.3200988 ,  0.526784  , -0.7703731 ]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-1.3080608 , -0.13253094,  0.5513761 ], dtype=float32)>


In [6]:
def hypothesis(X):
  return tf.nn.softmax(tf.matmul(x_data, W) + b)

print(hypothesis(x_data))

Instructions for updating:
Colocations handled automatically by placer.
tf.Tensor(
[[1.3657196e-02 7.9016238e-03 9.7844124e-01]
 [3.9259776e-02 1.7034741e-02 9.4370544e-01]
 [3.8038525e-01 1.6772316e-01 4.5189157e-01]
 [3.2339048e-01 5.9075940e-02 6.1753362e-01]
 [3.6299741e-06 6.2072729e-08 9.9999630e-01]
 [2.6252020e-02 1.0727962e-02 9.6301997e-01]
 [1.5652511e-05 4.2180278e-07 9.9998391e-01]
 [2.9407690e-06 3.8113328e-08 9.9999702e-01]], shape=(8, 3), dtype=float32)


In [7]:
# Softmax onehot test
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print(hypothesis(sample_db))

tf.Tensor(
[[1.3657196e-02 7.9016238e-03 9.7844124e-01]
 [3.9259776e-02 1.7034741e-02 9.4370544e-01]
 [3.8038525e-01 1.6772316e-01 4.5189157e-01]
 [3.2339048e-01 5.9075940e-02 6.1753362e-01]
 [3.6299741e-06 6.2072729e-08 9.9999630e-01]
 [2.6252020e-02 1.0727962e-02 9.6301997e-01]
 [1.5652511e-05 4.2180278e-07 9.9998391e-01]
 [2.9407690e-06 3.8113328e-08 9.9999702e-01]], shape=(8, 3), dtype=float32)


In [8]:
def cost_fn(X, Y):
  logits = hypothesis(X)
  cost = -tf.reduce_sum(Y * tf.log(logits), axis=1)
  cost_mean = tf.reduce_mean(cost)
  return cost_mean

print(cost_fn(x_data, y_data))

tf.Tensor(6.07932, shape=(), dtype=float32)


In [9]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x # x^2
dy_dx = g.gradient(y, x) # Will compute to 6.0
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [10]:
def grad_fn(X, Y):
  with tf.GradientTape() as tape:
    cost = cost_fn(X, Y)
    grads = tape.gradient(cost, variables)
    return grads
  
print(grad_fn(x_data, y_data))

[<tf.Tensor: id=117, shape=(4, 3), dtype=float32, numpy=
array([[ 0.06914608, -0.6509784 ,  0.5818323 ],
       [-1.5221257 , -1.214863  ,  2.7369888 ],
       [-1.2473829 , -1.7611003 ,  3.008483  ],
       [-1.2014606 , -1.8659232 ,  3.0673838 ]], dtype=float32)>, <tf.Tensor: id=115, shape=(3,), dtype=float32, numpy=array([-0.15212913, -0.34219202,  0.4943211 ], dtype=float32)>]


In [12]:
def fit(X, Y, epochs=2000, verbose=100):
  optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
  for i in range(epochs):
    grads = grad_fn(X, Y)
    optimizer.apply_gradients(zip(grads, variables))
    if (i==0) | ((i+1)%verbose==0):
      print('Loss at epoch %d: %f' %(i+1, cost_fn(X,Y).numpy()))
      
fit(x_data, y_data)

Loss at epoch 1: 2.849417
Loss at epoch 100: 0.684151
Loss at epoch 200: 0.613813
Loss at epoch 300: 0.558204
Loss at epoch 400: 0.508306
Loss at epoch 500: 0.461059
Loss at epoch 600: 0.415072
Loss at epoch 700: 0.369636
Loss at epoch 800: 0.324533
Loss at epoch 900: 0.280721
Loss at epoch 1000: 0.246752
Loss at epoch 1100: 0.232798
Loss at epoch 1200: 0.221645
Loss at epoch 1300: 0.211476
Loss at epoch 1400: 0.202164
Loss at epoch 1500: 0.193606
Loss at epoch 1600: 0.185714
Loss at epoch 1700: 0.178415
Loss at epoch 1800: 0.171645
Loss at epoch 1900: 0.165350
Loss at epoch 2000: 0.159483


In [13]:
sample_data = [[2,1,3,2]] # answer_label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis(sample_data)

print(a)
print(tf.argmax(a, 1)) #index: 2

tf.Tensor(
[[2.1975954e-06 1.2331181e-03 9.9876475e-01]
 [1.1288547e-03 8.1546679e-02 9.1732454e-01]
 [2.2205445e-07 1.6418663e-01 8.3581316e-01]
 [6.3921329e-06 8.5045439e-01 1.4953929e-01]
 [2.6150796e-01 7.2644740e-01 1.2044536e-02]
 [1.3783240e-01 8.6214012e-01 2.7417585e-05]
 [7.4242127e-01 2.5754178e-01 3.6978578e-05]
 [9.2197531e-01 7.8024112e-02 6.0005914e-07]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


In [14]:
b = hypothesis(x_data)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1)) # matches with y_data

tf.Tensor(
[[2.1975954e-06 1.2331181e-03 9.9876475e-01]
 [1.1288547e-03 8.1546679e-02 9.1732454e-01]
 [2.2205445e-07 1.6418663e-01 8.3581316e-01]
 [6.3921329e-06 8.5045439e-01 1.4953929e-01]
 [2.6150796e-01 7.2644740e-01 1.2044536e-02]
 [1.3783240e-01 8.6214012e-01 2.7417585e-05]
 [7.4242127e-01 2.5754178e-01 3.6978578e-05]
 [9.2197531e-01 7.8024112e-02 6.0005914e-07]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
